In [29]:
import requests 
import xml.etree.ElementTree as ET 
import urllib.request
from xml.dom import minidom
import zipfile
import os
from xml.etree.ElementTree import XML
import csv
import boto3
from botocore.exceptions import NoCredentialsError

In [ ]:
def download_first_xml(link="https://registers.esma.europa.eu/solr/esma_registers_firds_files/select?q=*&fq=publication_date:%5B2020-01-08T00:00:00Z+TO+2020-01-08T23:59:59Z%5D&wt=xml&indent=true&start=0&rows=100"):
    """ download_first_xml takes link of xml and downloads xml into feed.xml"""
    response = requests.get(link)
    with open('feed.xml', 'wb') as file:
        file.write(response.content)
    file.close()

In [ ]:
def read_first_xml_provide_url(path):
    """ read_first_xml_provide_url takes path and name of xml and parses it to find the download url of first file with DLTINS filetype """
    itemlist = minidom.parse(path).getElementsByTagName('response')
    for f in itemlist:
        itemlist1 = f.getElementsByTagName('doc')      
    for i in itemlist1:
        print(i.getElementsByTagName('str').item(6).attributes['name'].value )
        print(i.getElementsByTagName('str').item(6).firstChild.data)
        if(i.getElementsByTagName('str').item(6).firstChild.data=='DLTINS'):
            print('get the link')
            return(i.getElementsByTagName('str').item(1).firstChild.data)

In [ ]:
def download_zip(download_url):
    """ download_zip takes download_url and downloads the zip file xml_inside.zip """
    read = requests.get(download_url, stream = True,allow_redirects=True) 
    with open("xml_inside.zip","wb") as xml_inside_zip: 
        for chunk in read.iter_content(chunk_size=1024): 
            if chunk: 
                xml_inside_zip.write(chunk) 

In [ ]:
def unzip_getxml():
    """ unzip_getxml extracts the xml file from  xml_inside.zip into targetdir folder"""    
    with zipfile.ZipFile("xml_inside.zip","r") as zip_ref:
        zip_ref.extractall("targetdir")

In [ ]:

ACCESS_KEY = 'XXXXXXXXXXXXXXXXXXXXXXX'
SECRET_KEY = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'


def upload_to_aws(local_file, bucket, s3_file):
    """ upload_to_aws takes local file path and name bucket name and s3_file name and uploads the file into s3 bucket """  
    s3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY,
                      aws_secret_access_key=SECRET_KEY)
    try:
        s3.upload_file(local_file, bucket, s3_file)
        print("Upload Successful")
        return True
    except FileNotFoundError:
        print("The file was not found")
        return False
    except NoCredentialsError:
        print("Credentials not available")
        return False



In [ ]:
def xml_tocsv_uploadaws(zippath='targetdir'):
    """ xml_tocsv_uploadaws takes zippath and checks the extracted xmls to convert to csv files and upload to s3 bucket """  
    path = zippath

    for filename in os.listdir(path):
        print(filename)
        # Instead of using pandas we are using csv library by referring to code at 
        #https://medium.com/analytics-vidhya/converting-xml-data-to-csv-format-using-python-3ea09fa18d38
        csv_file=open(filename+'.csv','w',newline='',encoding='utf-8')
        csv_cursor=csv.writer(csv_file)
        col_names=['FinInstrmGnlAttrbts.Id',    'FinInstrmGnlAttrbts.FullNm',    'FinInstrmGnlAttrbts.ClssfctnTp',
        'FinInstrmGnlAttrbts.CmmdtyDerivInd',    'FinInstrmGnlAttrbts.NtnlCcy',    'Issr']
        csv_cursor.writerow(col_names)
        tree = ET.parse(path+'/'+filename)
        root = tree.getroot()
        for BizData in root:
            for Pyld in BizData:
                for Document in Pyld:
                    for FinInstrmRptgRefDataDltaRpt in Document:
                        for FinInstrm in FinInstrmRptgRefDataDltaRpt:
                            for ModfdRcrd in FinInstrm:
                           #     print(ModfdRcrd)
                                if ModfdRcrd.tag[48:]=='Issr':
                                    Issr=attrib.text
                                    IssrFound=True
                     #               print(Issr,'Issr========')
                                    if IdFound and FullNmFound and ClssfctnTpFound and CmmdtyDerivIndFound and IssrFound:
                                    #    print(Id,IdFound,FullNm,FullNmFound,ClssfctnTp,ClssfctnTpFound,CmmdtyDerivInd,CmmdtyDerivIndFound,NtnlCcy,NtnlCcyFound)
                                        csv_cursor.writerow([Id,FullNm,ClssfctnTp,CmmdtyDerivInd,NtnlCcy,Issr])
                                        del Id
                                        del IdFound
                                        del FullNm
                                        del FullNmFound
                                        del ClssfctnTp
                                        del ClssfctnTpFound
                                        del CmmdtyDerivInd
                                        del CmmdtyDerivIndFound
                                        del NtnlCcy
                                        del NtnlCcyFound
                                        del Issr
                                        del IssrFound
                                IdFound = FullNmFound = ClssfctnTpFound = CmmdtyDerivIndFound = NtnlCcyFound = IssrFound=False
                                for attrib in list(ModfdRcrd):

                                  #  print(attrib.tag[48:],attrib.text)
                                    tag=attrib.tag[48:]
                                    if tag=='Id':
                                        Id=attrib.text
                              #          print(Id," Id")
                                        IdFound=True
                                    if tag=='FullNm':
                                        FullNm=attrib.text
                               #         print(FullNm,'====FullNm')
                                        FullNmFound=True
                                    if tag=='ClssfctnTp':
                                        ClssfctnTp=attrib.text
                                        ClssfctnTpFound=True
                                #        print(ClssfctnTp,'======ClssfctnTp')
                                    if tag=='CmmdtyDerivInd':
                                        CmmdtyDerivInd=attrib.text
                                 #       print(CmmdtyDerivInd,'=====CmmdtyDerivInd')
                                        CmmdtyDerivIndFound=True
                                    if tag=='NtnlCcy':
                                        NtnlCcy=attrib.text
                                  #      print(NtnlCcy,'NtnlCcy==========')
                                        NtnlCcyFound=True
        csv_file.close()
        upload_to_aws(filename+'.csv', 'bucket_name', filename+'.csv')


In [ ]:
if __name__ == "__main__":
    try:
        download_first_xml()
    except Exception as e:
        print("download_first_xml    class:", e.__class__,"      message:" ,e.message,"     args:" ,e.args)
    try:
        download_url=read_first_xml_provide_url('feed.xml')
    except Exception as e:
        print("download_first_xml    class:", e.__class__,"      message:" ,e.message,"     args:" ,e.args)
    try:
        download_zip(download_url)
    except Exception as e:
        print("download_first_xml    class:", e.__class__,"      message:" ,e.message,"     args:", e.args)
    try:
        unzip_getxml()
    except Exception as e:
        print("download_first_xml    class:", e.__class__,"      message:" ,e.message,"     args:" ,e.args)
    try:
        xml_tocsv_uploadaws()
    except Exception as e:
        print("download_first_xml    class:", e.__class__,"      message:" ,e.message,"     args:", e.args)
